In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
# Activation function

# ReLU 함수와 softmax 함수
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_feature, n_hidden) # 은닉층
        self.relu = nn.ReLU(inplace=True)
        self.out = nn.Linear(n_hidden, n_output) # 출력층
        self.softmax = nn.Softmax(dim=n_output)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x) # 은닉층에 적용되는 ReLU 활성화 함수
        x = self.out(x)
        x = self.softmax(x) # 출력층에 적용되는 소프트맥스 활성화 함수
        return x

In [ ]:
# Loss function

# MSE(평균 제곱 오차)
loss_fn = nn.MSELoss(reduction='mean')
# reduction='mean' : 예측값과 실젯값의 차이의 평균을 loss로 사용한다는 의미
# reduction='sum' : 모든 샘플에 대한 loss를 합산하여 반환
# reduction='none' : 각 샘플에 대한 loss를 그대로 반환
y_pred = model(x)
loss = loss_fn(y_pred, y)

# 크로스 엔트로피 오차
loss = nn.CrossEntropyLoss()
input = torch.randn(5, 6, requires_grad=True)
# 평균 0, 표준편차 1의 가우시안 정규분포를 이용해 5X6의 난수 텐서 생성
# requires_grad=True : 이 텐서에 대해 그래디언트 계산 가능
target = torch.empty(5, dtype=torch.long).random_(6)
# torch.empty(5) : 길이가 5인 텐서 생성 (쓰레기 값 가짐)
# .random_(6) : 이 텐서에 0~5의 난수 할당 (6개의 클래스 중에서 5개의 레이블 무작위 선택)
# (target = torch.randint(0, 6, 5, dtype=torch.long))
output = loss(input, target)
output.backward()

In [6]:
# Avoid overfitting

# Dropout
class DropoutModel(nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()
        self.layer1 = nn.Linear(784, 1200)
        self.dropout1 = nn.Dropout(0.5) # 50%의 노드를 무작위로 선택해 사용하지 않겠다는 의미
        self.layer2 = nn.Linear(1200, 1200)
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(1200,10)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        return self.layer3(x)

In [ ]:
# Improve performance

# Mini-batch gradient descent
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        self.y_data = [[12], [18], [11]]

        def __len__(self):
            return len(self.x_data)

        def __getitem__(self, idx):
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x, y

dataset = CustomDataset()
dataloader = DataLoader(
    dataset,
    batch_size=2, # 미니 배치 크기로 2의 제곱수 사용
    shuffle=True
)